In [1]:
# Workflow Agents in Google ADK: Itinerary-Driven Packing List Example
# Welcome 👋🏻 In this notebook, you'll learn how to use Workflow Agents in Google's Agent Development Kit (ADK)
# to chain together two LLM agents: one that creates a travel itinerary, and another that generates a packing list
# based on that itinerary. This mirrors how real travelers plan: first decide what you'll do, then pack accordingly!

In [2]:
# Install Google ADK for Python
# This is the foundational package that provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

Name: google-adk
Version: 1.21.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: /opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages
Requires: aiosqlite, anyio, authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-bigquery, google-cloud-bigquery-storage, google-cloud-bigtable, google-cloud-discoveryengine, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-storage, google-genai, graphviz, jsonschema, mcp, opentelemetry-api, opentelemetry-exporter-gcp-logging, opentelemetry-exporter-gcp-monitoring, opentelemetry-exporter-gcp-trace, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, opentelemetry-sdk, pyarrow, pydantic, python-dateutil, python-dotenv, PyYAML, requests, sqlalchemy, sqlalchemy-spanner, starlette, tenacity, typing-extensions, tzlocal, uvicorn, watchdog

In [4]:
# Configure environment
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Load API key from .env file
# Create a .env file with: GOOGLE_GENAI_API_KEY=your-api-key-here
api_key = os.environ.get("GOOGLE_GENAI_API_KEY")

if not api_key:
    raise ValueError(
        "GOOGLE_GENAI_API_KEY not found.\n"
        "Create a .env file with: GOOGLE_GENAI_API_KEY=your-api-key-here\n"
        "Get your API key from: https://aistudio.google.com/apikey"
    )

os.environ["GEMINI_API_KEY"] = api_key

print(f"✓ API key configured (length: {len(api_key)} characters)")
print(f"✓ GOOGLE_GENAI_USE_VERTEXAI = {os.environ.get('GOOGLE_GENAI_USE_VERTEXAI')}")

# Define the specific Gemini model we'll use. 
# 'gemini-2.0-flash' is a fast and efficient model.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

✓ API key configured (length: 39 characters)
✓ GOOGLE_GENAI_USE_VERTEXAI = False


In [5]:
# Import Necessary Modules

# Agent: The fundamental class for creating an AI agent within ADK.
from google.adk.agents import Agent

# SequentialAgent: A specialized agent that chains together multiple sub-agents in a predefined sequence.
from google.adk.agents import SequentialAgent

# Runner: The orchestrator that manages the flow of messages and agent execution.
from google.adk.runners import Runner

#  InMemorySessionService: A simple, in-memory service for managing conversation sessions. 
# Sessions are vital for maintaining context in multi-turn dialogues.
from google.adk.sessions import InMemorySessionService

# google_search: An ADK tool that provides agents with the ability to perform web searches, enabling them to retrieve up-to-date information.
from google.adk.tools import google_search

# types from google.genai: Provides data structures (like Content and Part) to represent messages exchanged with the language model.
from google.genai import types

# IPython.display: Used for rendering rich content like Markdown in Jupyter notebooks.
from IPython.display import Markdown, display

In [6]:
# Itinerary Agent: Definition
# This agent is designed to create a detailed travel itinerary for a given destination and duration.
# It leverages the `google_search` tool to find up-to-date recommendations for attractions, restaurants, and activities.
# The `output_key="itinerary"` specifies that this agent's primary output will be accessible under the key "itinerary",
# which is crucial for passing this output to subsequent agents in a workflow.
itinerary_agent = Agent(
    name="itinerary_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that creates a travel itinerary for a given destination and trip duration, using Google Search for up-to-date recommendations.",
    instruction="""
        You are a helpful and creative travel itinerary planner.
        When the user provides a destination and trip duration, use the [google_search] tool to find current and popular attractions, restaurants, and activities for each day.
        For each day, create a detailed schedule with 2-4 activities, including at least one meal suggestion and one local attraction, prioritizing the user's interests (such as art and food).
        Present the itinerary in a clear, easy-to-read markdown format, organized by day.

        Example format:

        ### Day 1
        - Morning: [Attraction or activity]
        - Lunch: [Restaurant or food experience]
        - Afternoon: [Attraction or activity]
        - Evening: [Dinner suggestion or event]

        ### Day 2
        ...

        Always use the [google_search] tool to find the latest recommendations for each activity or restaurant.
        When providing recommendations, if a direct and relevant official website or information page URL is prominently found in the search results for a specific item, always include it as a Markdown hyperlink like [Item Name](URL). try to add as many hyperlinks as possible!
        Be concise, friendly, and ensure the itinerary is complete for all days requested.
    """,
    tools=[google_search], # This is the crucial part: assigning the tool to the agent!
    output_key="itinerary"  # This agent's output will be stored with the key "itinerary".
)

print("`itinerary_agent` successfully defined with `google_search` tool.")

`itinerary_agent` successfully defined with `google_search` tool.


In [7]:
# Packing List Agent: Definition
# This agent's primary role is to generate a comprehensive packing list.
# Crucially, its instruction includes a placeholder `{itinerary}`, which tells the agent to
# expect and use the output from a previous step (the itinerary_agent in this workflow).
# The `output_key="packing_list"` ensures that its final output is stored under this key.
packing_list_agent = Agent(
    name="packing_list_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Generates a packing list based on itinerary, destination, and user preferences.",
    instruction="""
        You are a helpful travel assistant.
        The user will provide their destination, trip duration, and a detailed itinerary (including planned activities, locations, and types of events for each day).
        Your task is to generate a complete, practical packing list tailored to the specific activities and locations in the itinerary.
        
        Itinerary: {itinerary} 

        **Thoroughly analyze the provided Itinerary, considering the specific activities, locations, and time of day for each entry.** Determine necessary clothing, gear, and items based *directly* on these plans. For example, if the itinerary mentions hiking, suggest hiking boots. If it mentions a fancy dinner, suggest formal wear.

        Organize the packing list by category (such as Clothing, Toiletries, Electronics, Documents, and Other Essentials).
        If you notice a special activity or requirement in the itinerary, include specific items for it and justify their inclusion based on the activity.
        If any information is missing, make reasonable assumptions based on the itinerary and destination—do not ask the user for more details unless absolutely necessary.
        
        **After the main categorized list, provide a dedicated 'Outfit Suggestions for Key Activities' section.** For 2-3 prominent activities from the itinerary, suggest a complete outfit including clothing, shoes, and any relevant accessories, explaining *why* it's suitable for that specific event.

        Present the complete output in a clear, easy-to-read markdown format. For example:

        #### Clothing
        - Comfortable walking shoes (for city tours and museum visits)
        - Lightweight jacket (for potentially cooler evenings in [Destination])
        - Dress for dinner at a nice restaurant (if formal dining is in itinerary)
        - Swimwear (if beach or pool is in itinerary)
        - ...

        #### Toiletries
        - Toothbrush and toothpaste
        - Sunscreen (for outdoor activities)
        - ...

        #### Electronics
        - Phone and charger
        - Plug adapter (if needed for [Destination])
        - ...

        #### Documents
        - Passport
        - Travel insurance (important for any trip abroad)
        - ...

        #### Other Essentials
        - Daypack for city tours
        - Reusable water bottle

        #### Outfit Suggestions for Key Activities
        *   **Day 1 - Morning (Tokyo National Museum):** Comfortable smart casual. E.g., Dark jeans/trousers, a layered top (blouse or stylish t-shirt), and comfortable, stylish walking shoes. A light scarf or jacket for cooler museum interiors.
        *   **Day 2 - Evening (Ninja Restaurant):** Fun, relaxed evening wear. E.g., A casual dress or nice shirt with dark pants. Comfortable shoes for easy movement.
        *   **Day 3 - Afternoon (Sushi Making Class):** Practical and comfortable. E.g., T-shirt, comfortable pants that allow movement, and closed-toe shoes. Avoid overly loose sleeves for hygiene and safety.

        Only ask clarifying questions if the itinerary is extremely vague or unclear. Otherwise, use your best judgment and generate a full, actionable packing list.
    """,
    output_key="packing_list"  # This agent's output will be stored with the key "packing_list".
)

In [8]:
# Sequential Agent: Definition
# The SequentialAgent allows you to define a workflow by chaining multiple sub-agents together.
# In this setup, the output of the first sub-agent (itinerary_agent) is automatically
# passed as input to the instruction of the next sub-agent (packing_list_agent)
# through the `{itinerary}` placeholder defined in the packing_list_agent's instruction.
wanderwise_agent = SequentialAgent(
    name="wanderwise_agent",
    sub_agents=[itinerary_agent, packing_list_agent],  # Defines the order of execution.
    description="Plans a travel itinerary and prepares a packing list based on activities planned."
)

In [9]:
# Import necessary modules for type hinting
from typing import Dict

# Modular Function for Agent Interaction (Corrected and Updated for multiple outputs)
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> Dict[str, str]:
    """
    Runs an interaction with an ADK agent (can be a single Agent or SequentialAgent)
    and returns a dictionary of its final text responses.
    Each response is keyed by the agent's 'name' (event.agent_name),
    or a sequential 'response_N' if the agent_name is not available.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        A dictionary where keys are the `agent_name` of the agent that produced the final response.
        Values are the full text responses.
        Returns an empty dictionary if no final response events are found.
    """
    # Use the provided session service if available, otherwise create a new one.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent.
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    # Collect all final responses
    all_final_responses = []
    for event in events:
        if event.is_final_response():
            # Concatenate all parts in the final response
            full_text = ''.join(part.text for part in event.content.parts if part.text)
            all_final_responses.append(full_text)
    return all_final_responses

print("Modular function `run_adk_agent_interaction` successfully updated to use `event.agent_name` for keys.")

Modular function `run_adk_agent_interaction` successfully updated to use `event.agent_name` for keys.


In [10]:
# Set up the session and user input
APP_NAME = "wanderwise_app"
USER_ID = "default_user"
SESSION_ID = f"{APP_NAME}_default_session"

#  Create a single instance of the session service.
# Passing a shared instance (`common_session_service`) to `run_adk_agent_interaction`
# is good practice, especially if you had a database-backed session service.
# For InMemorySessionService, it ensures all sessions are managed within the same memory scope.
common_session_service = InMemorySessionService()

# Define a list of diverse travel planning test cases for our workflow agent.
# Each dictionary contains a descriptive label, and the user's input.
test_cases = [
    {
        "label": "Adventure & Nature Trip (Banff)",
        "input_text": "I'm planning a 7-day trip to Banff National Park in August. I want to do moderate hiking and wildlife photography.",
    },
    {
        "label": "Culture & History Trip (Rome)",
        "input_text": "I'm a 35 yo male, and I am going to Rome for 5 days in October. I'm very interested in ancient history, classical art, and local cuisine.",
    },
    {
        "label": "Beach & Relaxation Trip (Maldives)",
        "input_text": "I'm visiting the Maldives for 4 days in January for relaxation and some water activities. 25F.",
    },
    {
        "label": "City Break & Shopping (Paris)",
        "input_text": "I'm doing a 3-day city break in Paris in April. I want to visit famous landmarks, do some shopping, and enjoy cafes. Love Emily in Paris!",
    }
]

print("------ [starting workflow agent interactions] ------\n")

# Iterate through each test case and run the travel planning workflow.
# The 'wanderwise_agent' is the SequentialAgent which orchestrates the 
# 'itinerary_agent' followed by the 'packing_list_agent'.
# The 'run_adk_agent_interaction' helper function handles session creation and
# collects all final responses (itinerary and packing list) into a list.
for i, case in enumerate(test_cases):
    print(f"\n--- Example {i+1}: {case['label']} ---")
    print(f"User Input: \"{case['input_text']}\"")

    # Invoke the workflow agent using the modular helper function.
    # We pass the shared 'common_session_service' instance.
    workflow_outputs_list = await run_adk_agent_interaction(
        agent=wanderwise_agent,
        user_id="user"+f"_{i+1}",
        session_id="session"+f"_{i+1}",  # Unique session ID for each test case
        input_text=case["input_text"],
        app_name=APP_NAME,
        session_service=common_session_service # Pass the shared service instance
    )

    # Parse and Display Outputs from the list.
    if workflow_outputs_list:
        for j, text_response in enumerate(workflow_outputs_list):
            display(Markdown(text_response))
    else:
        print(f"No outputs received for Example {i+1}.")
    
    print(f"--- Example {i+1}: Workflow Agent Interaction Complete ---")


print("\n------ [all workflow agent interactions complete] ------")


------ [starting workflow agent interactions] ------


--- Example 1: Adventure & Nature Trip (Banff) ---
User Input: "I'm planning a 7-day trip to Banff National Park in August. I want to do moderate hiking and wildlife photography."


Okay! Here is a possible itinerary for your 7-day trip to Banff National Park in August, focusing on moderate hiking and wildlife photography:

### Day 1: Arrival in Banff and Vermillion Lakes

*   **Afternoon:** Arrive at Calgary International Airport (YYC) and transfer to Banff. Check into your accommodation.
*   **Late Afternoon:** Head to Vermillion Lakes, a short drive from the town of Banff, to capture reflections of Mount Rundle, especially beautiful during sunset. This is a great spot for wildlife photography, with opportunities to see elk and bald eagles.
*   **Dinner:** Enjoy a meal at [Banff Ave Brewing Co.](https://www.banffavebrewingco.ca/) for a taste of local craft beer and pub fare.

### Day 2: Johnston Canyon and Bow Valley Parkway

*   **Morning:** Hike the Johnston Canyon Trail to the Lower and Upper Falls. This is a popular, moderate hike with beautiful canyon views.
*   **Lunch:** Have lunch at the Blackswift Bistro in Johnston Canyon.
*   **Afternoon:** Drive along the Bow Valley Parkway (Highway 1A) for wildlife viewing opportunities. Keep an eye out for elk, deer, and possibly bears. Stop at scenic viewpoints like [Morant's Curve](https://www.pc.gc.ca/en/lhn-nhs/ab/castle/decouvrir-discover/morant) for the classic train photo.
*   **Dinner:** Dine at [The Bison Restaurant & Terrace](https://www.banffhospitalitycollective.com/restaurants/the-bison/) for seasonal Canadian cuisine.

### Day 3: Lake Louise and Plain of Six Glaciers

*   **Morning:** Drive to Lake Louise and take in the iconic views of the lake and the Fairmont Chateau Lake Louise.
*   **Morning/Afternoon:** Hike the Plain of Six Glaciers Trail, a moderate 10.6 km round trip that offers stunning views of the lake and glaciers. Enjoy tea and snacks at the Plain of Six Glaciers Tea House.
*   **Dinner:** Enjoy a lakeside dinner at the [Lakeview Lounge](https://www.fairmont.com/lake-louise/dining/lakeview-lounge/) at the Fairmont Chateau Lake Louise.

### Day 4: Hiking near Sunshine Village

*   **Morning:** Take the gondola to Sunshine Village and hike to Healy Pass for wildflower viewing.
*   **Lunch:** Have lunch at Whitehorn Bistro for incredible views.
*   **Afternoon:** Hike the Howard Douglas Lake – Citadel Pass trail.
*   **Dinner:** Treat yourself to a memorable fondue experience at [The Grizzly House](https://www.thegrizzlyhouse.com/).

### Day 5: Icefields Parkway and Bow Lake

*   **Morning:** Drive north along the Icefields Parkway (Highway 93 N). This scenic drive offers numerous opportunities for wildlife sightings and photography.
*   **Morning/Afternoon:** Stop at Bow Lake, and hike to Bow Glacier Falls.
*   **Dinner:** Dine at [Sky Bistro](https://www.banffjaspercollection.com/dining/sky-bistro/) at the top of the Banff Gondola for amazing views.

### Day 6: Sulphur Mountain and Banff Town

*   **Morning:** Hike up Sulphur Mountain for panoramic views of the Bow Valley. Alternatively, take the Banff Gondola.
*   **Lunch:** Grab a quick bite at a cafe in Banff town. [Whitebark Cafe](https://whitebarkcafe.com/) is a good option.
*   **Afternoon:** Explore the town of Banff, visit the Whyte Museum of the Canadian Rockies, and shop for souvenirs.
*   **Dinner:** Enjoy a meal at [Park Distillery Restaurant + Bar](https://www.banffhospitalitycollective.com/restaurants/park-distillery/).

### Day 7: Departure

*   **Morning:** Enjoy a final breakfast in Banff.
*   **Morning/Afternoon:** Depending on your flight schedule, consider a short hike near Banff, such as the Fenland Trail.
*   **Afternoon:** Transfer back to Calgary International Airport (YYC) for your departure.

Have an amazing trip!


Okay, here is a detailed packing list tailored for your 7-day hiking and photography trip to Banff National Park in August, based on the provided itinerary.

#### Clothing
*   **Hiking boots:** Sturdy, broken-in hiking boots are essential for the various trails.
*   **Hiking socks:** Moisture-wicking hiking socks to prevent blisters.
*   **Base layers:** Moisture-wicking shirts (both short and long-sleeved) to regulate body temperature.
*   **Insulating layer:** A fleece jacket or a lightweight down jacket for warmth, especially in the evenings and at higher altitudes.
*   **Waterproof and windproof jacket:** Essential for unpredictable mountain weather.
*   **Hiking pants:** Quick-drying, convertible hiking pants are ideal.
*   **Casual pants/jeans:** For evenings in town and casual dining.
*   **T-shirts:** For layering and everyday wear.
*   **Long-sleeved shirts:** For sun protection and layering.
*   **Warm hat:** Beanie or toque for cooler evenings and higher elevations.
*   **Gloves or mittens:** Lightweight gloves for chilly mornings or evenings.
*   **Sun hat:** Wide-brimmed hat for sun protection during hikes.
*   **Underwear:** Moisture-wicking underwear is recommended for hiking.
*   **Sleepwear:** Comfortable pajamas or sleepwear.
*   **Dressy casual outfit:** For dinners at restaurants like The Bison or Sky Bistro (nice pants/skirt and a collared shirt/blouse).

#### Toiletries
*   **Sunscreen:** High SPF, broad-spectrum sunscreen.
*   **Lip balm with SPF:** To protect against sun and wind.
*   **Insect repellent:** To ward off mosquitoes and other bugs.
*   **After-bite relief:** For any insect bites you may get.
*   **Shampoo, conditioner, and soap:** Travel-sized toiletries.
*   **Toothbrush, toothpaste, and floss:**
*   **Deodorant:**
*   **Any personal medications:** With copies of prescriptions.
*   **Hand sanitizer:**
*   **Wet wipes:** For quick clean-ups on the trail.
*   **Towel:** Quick-drying travel towel.

#### Electronics
*   **Camera:** Essential for wildlife and landscape photography.
*   **Camera lenses:** Wide-angle, telephoto, and any specialty lenses.
*   **Extra camera batteries:** Cold weather can drain batteries quickly.
*   **Memory cards:** Bring plenty of memory cards for your camera.
*   **Phone and charger:**
*   **Portable power bank:** To recharge devices on the go.
*   **Headphones:**
*   **Binoculars:** For wildlife viewing.

#### Documents
*   **Passport/ID:**
*   **Flight/travel confirmations:**
*   **Travel insurance information:**
*   **Park pass:** If you plan to visit multiple parks, consider a Parks Canada Discovery Pass.

#### Other Essentials
*   **Daypack:** A comfortable daypack (20-30L) for hiking.
*   **Reusable water bottles or hydration reservoir:** Staying hydrated is crucial.
*   **Headlamp or flashlight:** For early morning or late evening hikes.
*   **Bear spray:** Essential for hiking in bear country; know how to use it.
*   **Bear bell:** To alert bears to your presence.
*   **First-aid kit:** Including blister treatment, pain relievers, antiseptic wipes, and bandages.
*   **Sunglasses:** With UV protection.
*   **Snacks:** Energy bars, nuts, dried fruit, etc., for hiking.
*   **Cash and credit cards:**
*   **Repair kit:** Duct tape, knife/multi-tool, cable ties.
*   **Trekking poles:** Helpful for stability on steep trails.
*   **Dry bags or Ziploc bags:** To protect electronics and valuables from rain.
*   **Book/entertainment:** For downtime.

#### Outfit Suggestions for Key Activities

*   **Day 3 - Hiking Plain of Six Glaciers:**
    *   **Base Layer:** Moisture-wicking t-shirt.
    *   **Mid Layer:** Fleece jacket for warmth.
    *   **Outer Layer:** Waterproof/windproof jacket.
    *   **Pants:** Convertible hiking pants.
    *   **Footwear:** Broken-in hiking boots with hiking socks.
    *   **Accessories:** Sun hat, sunglasses, backpack with water, snacks, sunscreen, bear spray.
    *   *Reasoning:* This outfit allows for layering to adjust to changing weather conditions and provides protection from the elements during a moderate hike.
*   **Day 5 - Icefields Parkway Wildlife Viewing & Bow Glacier Falls Hike:**
    *   **Base Layer:** Long-sleeved moisture-wicking shirt.
    *   **Mid Layer:** Down jacket for warmth during stops.
    *   **Outer Layer:** Waterproof/windproof jacket.
    *   **Pants:** Hiking pants.
    *   **Footwear:** Hiking boots.
    *   **Accessories:** Binoculars, camera with telephoto lens, bear spray, hat, gloves.
    *   *Reasoning:* Designed for a day of both driving and short hiking, with a focus on wildlife viewing. The layers provide warmth, while the outer shell protects from wind and rain. Binoculars and a telephoto lens are essential for spotting and photographing wildlife.
*   **Day 6 - Dinner at Park Distillery Restaurant + Bar:**
    *   **Top:** A nice button-down shirt or blouse.
    *   **Bottom:** Dark jeans or chinos.
    *   **Outerwear:** A stylish jacket or sweater.
    *   **Footwear:** Clean and comfortable shoes or boots.
    *   *Reasoning:* This outfit is suitable for a relaxed yet stylish dinner in a casual restaurant.


--- Example 1: Workflow Agent Interaction Complete ---

--- Example 2: Culture & History Trip (Rome) ---
User Input: "I'm a 35 yo male, and I am going to Rome for 5 days in October. I'm very interested in ancient history, classical art, and local cuisine."


Okay, I can help you plan a 5-day trip to Rome in October, focusing on ancient history, classical art, and local cuisine. Here is a possible itinerary:

Ok, here is a possible itinerary for your 5-day trip to Rome in October:

**Weather in Rome in October:**

October is a great time to visit Rome, with mild and pleasant weather. The average high temperature is around 20°C (68°F) during the day, and the average low is around 15°C (59°F) at night. You can expect about 7 hours of sunshine per day. There is a chance of rain, with an average of 10 rainy days during the month. It's a good idea to pack layers, including a light jacket or sweater for the evenings.

### Day 1: Ancient Rome & Local Flavors

*   **Morning:** Colosseum and Roman Forum. Explore the iconic Colosseum, imagining the gladiatorial contests and public spectacles that once took place there. Afterwards, wander through the ruins of the Roman Forum, the heart of ancient Rome's political, religious, and social life. Consider a guided tour to enhance your understanding of these historical sites.
*   **Lunch:** *Mordi e Vai*. Head to the Testaccio Market and grab a delicious Roman-style sandwich at Mordi e Vai. Try the *allesso di scottona con cicoria* (braised beef brisket with bitter greens) or other classic Roman fillings.
*   **Afternoon:** Palatine Hill. Climb Palatine Hill, one of the seven hills of Rome and legendary founding place of the city by Romulus. Enjoy panoramic views of the Roman Forum and Circus Maximus.
*   **Evening:** Food Tour in Trastevere. Immerse yourself in Roman cuisine with a food tour in the charming Trastevere neighborhood. Sample local specialties like *supplì*, pizza, pasta, and gelato while learning about the area's history and culture.

### Day 2: Vatican City & Artistic Wonders

*   **Morning:** Vatican Museums and Sistine Chapel. Visit the Vatican Museums, home to an extensive collection of art and artifacts spanning centuries. Marvel at Michelangelo's masterpiece, the Sistine Chapel ceiling, and explore the various galleries showcasing classical sculptures, Renaissance paintings, and more. Book your tickets in advance to avoid long lines.
*   **Lunch:** Near Vatican City. Grab a quick and tasty lunch at a local *pizza al taglio* (pizza by the slice) shop near Vatican City.
*   **Afternoon:** St. Peter's Basilica. Explore St. Peter's Basilica, one of the holiest shrines in Catholicism. Admire its grandeur and artwork, including Michelangelo's *Pietà*. Consider climbing to the top of the dome for breathtaking views of Rome.
*   **Evening:** *Ditirambo*. Enjoy dinner at Ditirambo, a restaurant near Campo de Fiori with high-quality Roman and Italian dishes and an extensive wine list.

### Day 3: Ancient Ostia

*   **Morning:** Travel to Ostia Antica. Take a train to Ostia Antica, the ancient port of Rome.
*   **Lunch:** Have lunch at a trattoria in the "borgo", near Ostia Antica.
*   **Afternoon:** Explore the ruins of Ostia Antica. Wander through the well-preserved ruins of this ancient city, imagining life in Roman times. See the remains of houses, shops, temples, and public buildings.
*   **Evening:** *La Tavernaccia Da Bruno*. Return to Rome and have dinner at La Tavernaccia Da Bruno, a restaurant in the Trastevere neighborhood.

### Day 4: Baroque Rome & Hidden Gems

*   **Morning:** Borghese Gallery and Gardens. Visit the Borghese Gallery, housing masterpieces by Bernini, Caravaggio, and Raphael. Explore the beautiful Borghese Gardens, enjoying the fall foliage. Book your tickets in advance, as entry is timed and limited.
*   **Lunch:** Enjoy lunch at a cafe in the Borghese Gardens.
*   **Afternoon:** Pantheon and Piazza Navona. Visit the Pantheon, an architectural marvel with its impressive dome and oculus. Afterwards, stroll through Piazza Navona, admiring the Fountain of Four Rivers and the lively atmosphere.
*   **Evening:** *Ai Tre Scalini*. Enjoy dinner at Ai Tre Scalini, a restaurant in the Monti neighborhood.

### Day 5: Appian Way & Departure

*   **Morning:** Appian Way and Catacombs. Explore the ancient Appian Way, one of the earliest Roman roads. Visit the Roman Catacombs, underground burial sites offering a glimpse into early Christian history.
*   **Lunch:** Have lunch near the Appian Way.
*   **Afternoon:** Depending on your departure time, you can visit any sites that you may have missed.
*   **Evening:** Depart from Rome.

Enjoy your trip to Rome!


...

Okay, here is a packing list tailored for your 4-day relaxing Maldives trip focusing on water activities, keeping in mind the warm January weather:

#### Clothing
*   Swimsuits (at least 2-3, quick-drying material recommended)
*   Swimsuit cover-up (sarong, light dress, or tunic)
*   Lightweight, breathable dresses/skirts (2-3 for evenings and casual daytime)
*   Shorts (2-3 pairs, linen or cotton)
*   T-shirts/tank tops (4-5, moisture-wicking fabric is a plus)
*   One nicer outfit for the fine dining experience (e.g., a dress or dressy top and pants)
*   Light sweater or cardigan (for potentially breezy evenings or air-conditioned spaces)
*   Underwear and socks
*   Sleepwear

#### Footwear
*   Flip-flops or sandals (for everyday wear)
*   Water shoes (for snorkeling and water sports, protecting your feet from coral)
*   Dressier sandals or shoes (for dinner)

#### Toiletries
*   Reef-safe sunscreen (high SPF, very important for protecting your skin and the coral reefs)
*   After-sun lotion or aloe vera gel
*   Insect repellent (especially for evenings)
*   Shampoo, conditioner, and body wash (travel-sized or solid versions)
*   Deodorant
*   Toothbrush, toothpaste, and floss
*   Makeup (if desired, keep it light)
*   Hairbrush and hair ties
*   Any personal prescription medications

#### Electronics
*   Phone and charger
*   Waterproof phone case or dry bag (for protecting your phone during water activities)
*   Camera or GoPro (for underwater photos and videos)
*   Chargers for all devices
*   Portable power bank (optional, for charging on the go)

#### Documents
*   Passport
*   Flight confirmation
*   Hotel/resort confirmation
*   Travel insurance information
*   Copies of important documents (stored separately from originals)

#### Other Essentials
*   Snorkel and mask (if you prefer your own, otherwise resorts usually provide them)
*   Sunglasses
*   Hat or visor (for sun protection)
*   Beach bag or tote
*   Reusable water bottle
*   Dry bag (for keeping valuables safe on boat trips)
*   Small backpack or daypack (for carrying essentials on excursions)
*   Cash (USD is widely accepted, small denominations are useful)
*   Any books or entertainment for relaxation

#### Outfit Suggestions for Key Activities

*   **Day 2 - Morning Snorkeling:** Swimsuit, rash guard (for extra sun protection), water shoes, snorkel and mask (if bringing your own), reef-safe sunscreen, and a waterproof phone case to take pictures.
*   **Day 2 - Evening Fine Dining:** A lightweight, elegant dress or a dressy top with tailored pants or a skirt. Dressy sandals or heels. Minimal jewelry and a small clutch.
*   **Day 3 - Afternoon Island Visit:** Comfortable shorts and a t-shirt or tank top. Comfortable walking sandals. A hat and sunglasses for sun protection. A lightweight backpack to carry water, sunscreen, and your wallet.


--- Example 3: Workflow Agent Interaction Complete ---

--- Example 4: City Break & Shopping (Paris) ---
User Input: "I'm doing a 3-day city break in Paris in April. I want to visit famous landmarks, do some shopping, and enjoy cafes. Love Emily in Paris!"


Okay, here is a possible 3-day itinerary for your city break in Paris in April!

### Day 1

*   **Morning:** Visit the Eiffel Tower. You can book tickets in advance to avoid long queues.
*   **Lunch:** Have lunch at [Café de l'Homme](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH4hOvKTPlVCwfEMc-xujgh71y10oI2rTK_0DBYxFqYlsDT_5EzDP4yJSF4ACGCs3HTRDLbM63jo1ls2hAAAgTEr03Ws2WZg8LGT4xwlVnT5Uh93Gk_yZR9yFOYrnOMcLKTV0XjN-7YPDyXxa2QsmM=) for exquisite French cuisine and breathtaking views of the Eiffel Tower.
*   **Afternoon:** Explore the Louvre Museum. Be sure to see the Mona Lisa.
*   **Evening:** Stroll along the Seine River and enjoy a Seine River cruise at sunset. For dinner, consider [Le Café Marly](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEee3lmVmppgIS7llz9xWlJ9HTyXcvVTyhohxLu2BvRL3ooqYURAKvuvMS1_8COI1IytcBHnTblKNNl2Wx18CHZH81k6GStV3KQzL-hYiLXlabGbb_jrh2XfDVgbBdESeJbbvAGqcTxsmfdSGcNoUPIUPMRDgh9DklGn7lqQ0SWfq0dqsA=) for a meal with a view of the Louvre.

### Day 2

*   **Morning:** Visit Notre Dame Cathedral. Afterwards, explore Île de la Cité, and visit Sainte-Chapelle.
*   **Lunch:** Enjoy lunch at [Bistrot Valois](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEee3lmVmppgIS7llz9xWlJ9HTyXcvVTyhohxLu2BvRL3ooqYURAKvuvMS1_8COI1IytcBHnTblKNNl2Wx18CHZH81k6GStV3KQzL-hYiLXlabGbb_jrh2XfDVgbBdESeJbbvAGqcTxsmfdSGcNoUPIUPMRDgh9DklGn7lqQ0SWfq0dqsA=), a lunch spot Emily frequents in the "Emily in Paris" series.
*   **Afternoon:** Shopping in Le Marais. This area has a mix of designer shops and vintage boutiques.
*   **Evening:** Experience Montmartre and visit the Sacré-Cœur Basilica. For dinner, try [La Maison Rose](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEPBNV1DpVX2RaagDT8v8JSscFQZWhzUPlWZu6wfOxsnu1txRALm8cN_D9RVBiauX2HXZL83ormYtwJPn7H18zRxBiy8rYKfyRnqe_t37UujrXdb-q_a9iBIPaLAqKRBnW26Qt0junb2Slm6kw0Mp378j6sox_0CwNo3RrfwDvHOIAKlKdKGwV6mjOtUPS39qRBdg=), a colorful and Instagram-worthy café featured in "Emily in Paris".

### Day 3

*   **Morning:** Visit the Palace of Versailles. Consider going on April 5th to see the Musical Fountains Show.
*   **Lunch:** Enjoy a delightful brunch at [Les Deux Magots](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFN6KHwvEQGZn3Zdc3ra9a595WQUhYhyGM5TeVdfmXWo1AIdh55rhXgptmAxmb7h23BuOx04qP_4pMbbTYN0y7C-Pd5T3g1atY56N_d4IOkyCehG7vv1JRj4KamXY0HqtWYw8S-UYqmYyADn4yHi_ZDnJ3WrWYg), a famous brasserie.
*   **Afternoon:** Shopping on the Champs-Élysées for luxury brands. Also, explore the Tuileries Garden.
*   **Evening:** Enjoy the view from the Arc de Triomphe. Dine at [Café de Flore](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEPBNV1DpVX2RaagDT8v8JSscFQZWhzUPlWZu6wfOxsnu1txRALm8cN_D9RVBiauX2HXZL83ormYtwJPn7H18zRxBiy8rYKfyRnqe_t37UujrXdb-q_a9iBIPaLAqKRBnW26Qt0junb2Slm6kw0Mp378j6sox_0CwNo3RrfwDvHOIAKlKdKGwV6mjOtUPS39qRBdg=), a legendary café.

Enjoy your trip to Paris!


Okay, here is a packing list for your 3-day Paris city break in April, keeping in mind the planned activities, locations, and "Emily in Paris" inspiration:

#### Clothing

*   **Tops:** 4-5 versatile tops (mix of blouses, t-shirts, and a dressy top)
*   **Bottoms:** 2 pairs of stylish pants or jeans, 1 skirt
*   **Sweater/Cardigan:** 1-2 lightweight sweaters or cardigans for layering.
*   **Jacket:** A fashionable trench coat or a versatile jacket suitable for spring weather.
*   **Dress:** A stylish dress suitable for evenings out or nicer lunches.
*   **Comfortable Walking Shoes:** Essential for all the walking you'll be doing! (Sneakers or stylish walking shoes)
*   **Flats/Loafers:** A pair of stylish flats or loafers for less strenuous activities.
*   **Heels/Dress Shoes:** A pair of heels or dress shoes if you plan on any fancier dinners.
*   **Socks:** Several pairs of comfortable socks.
*   **Underwear:** Enough for 3 days.
*   **Pajamas:** Comfortable sleepwear.

#### Toiletries

*   **Toothbrush, toothpaste, floss**
*   **Shampoo, conditioner, body wash** (travel-sized or refillable containers)
*   **Deodorant**
*   **Makeup:** As per your preference
*   **Makeup remover wipes or solution**
*   **Sunscreen:** Even in April, it's important to protect your skin.
*   **Moisturizer**
*   **Lip balm**
*   **Hairbrush/comb**
*   **Hair ties**
*   **Perfume/Cologne:**
*   **Hand sanitizer**
*   **Any personal prescription medications**

#### Electronics

*   **Phone and charger**
*   **Portable charger/power bank:** Useful for long days of sightseeing.
*   **Camera:** If you want higher-quality photos than your phone offers.
*   **Universal adapter:** If your devices use a different plug.

#### Documents

*   **Passport**
*   **Visa:** If required for your nationality.
*   **Flight/train tickets**
*   **Hotel reservation confirmations**
*   **Copies of important documents:** Keep separate from originals.
*   **Travel insurance information**
*   **Credit cards and cash**

#### Other Essentials

*   **Small backpack or crossbody bag:** For carrying essentials during the day.
*   **Reusable water bottle:** To stay hydrated during your tours.
*   **Sunglasses**
*   **Scarf:** A stylish scarf can add flair to your outfits and provide warmth.
*   **Umbrella:** April in Paris can be rainy.
*   **Small first-aid kit:** Band-aids, pain relievers, etc.
*   **Snacks:** For museum visits and long walks.
*   **Phrasebook or translation app:** Useful for communicating.
*   **Lipstick/lip gloss**
*   **Travel pillow**

#### Outfit Suggestions for Key Activities

*   **Day 1 - Afternoon (Louvre Museum):** Comfortable, stylish, and practical. For example, dark-wash jeans, a long-sleeved striped Breton top, a lightweight trench coat, comfortable walking shoes or fashionable sneakers, and a crossbody bag to keep your belongings secure.
*   **Day 2 - Evening (Montmartre/La Maison Rose):** Chic and Instagram-ready! E.g., A flowy midi skirt, a fitted top or blouse, a cardigan or light jacket, ankle boots or stylish flats, and a colorful scarf. Accessorize with a beret for a Parisian touch.
*   **Day 3 - Morning (Palace of Versailles):** Dressy Casual. E.g., A dress, tights (if needed, depending on the weather), a smart jacket or blazer, comfortable but stylish shoes like loafers or ballet flats, and a small, elegant handbag. Sunglasses and a hat can be useful if the Musical Fountains Show is outdoors.


--- Example 4: Workflow Agent Interaction Complete ---

------ [all workflow agent interactions complete] ------


In [11]:
# Congratulations 🎉 
# You’ve just built and run a SequentialAgent workflow in ADK that first creates an itinerary,
# then generates a packing list tailored to your actual plans.
# Try changing the user input or expanding the workflow with more steps!